In [21]:
%load_ext autoreload
%autoreload 2
import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from gymboard import GymBoard

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
seed = 42
gamma = 0.99
max_steps_per_episode = 10000
env = gym.make('CartPole-v0')
env.seed(seed)
eps = np.finfo(np.float32).eps.item()

In [23]:
num_inputs = 4
num_actions = 2
num_hidden = 128

inputs = layers.Input(shape=(num_inputs,))
common = layers.Dense(num_hidden, activation='relu')(inputs)
action = layers.Dense(num_actions, activation='softmax')(common)
critic = layers.Dense(1)(common)

model_actor = keras.Model(inputs=inputs, outputs=[action])
inputs_action = layers.Input(shape=(1,))
model_critic = keras.Model(inputs=[inputs, inputs_action], outputs=[critic])

In [24]:
optimizer_actor, optimizer_critic = keras.optimizers.Adam(learning_rate=0.01), keras.optimizers.Adam(learning_rate=0.01)
huber_loss = keras.losses.Huber()
action_probs_history = []
critic_value_history = []
rewards_history = []
running_reward = 0
episode_count = 0

gboard = GymBoard()
gboard.show()
gboard.write_env(env, model, step=0)

while True:  # Run until solved
# for _ in range(5):
    state = env.reset()
    episode_reward = 0
    with tf.GradientTape(persistent=True) as tape:
        for timestep in range(1, max_steps_per_episode):
            # env.render(); Adding this line would show the attempts
            # of the agent in a pop up window.

            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)

            # Predict action probabilities and estimated future rewards
            # from environment state
            action_probs = model_actor(state) 
            critic_value_history.append(critic_value[0, 0])

            # Sample action from action probability distribution
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            action_probs_history.append(tf.math.log(action_probs[0, action]))
            action = tf.expand_dims(action, 0)
                
            critic_value = model_critic([state, action])
            
            # Apply the sampled action in our environment
            state, reward, done, _ = env.step(action)
            rewards_history.append(reward)
            episode_reward += reward

            if done:
                break

        # Update running reward to check condition for solving
        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

        # Calculate expected value from rewards
        # - At each timestep what was the total reward received after that timestep
        # - Rewards in the past are discounted by multiplying them with gamma
        # - These are the labels for our critic
        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + gamma * discounted_sum
            returns.insert(0, discounted_sum)

        # Normalize
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
        returns = returns.tolist()

        # Calculating loss values to update our network
        history = zip(action_probs_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []
        for log_prob, value, ret in history:
            # At this point in history, the critic estimated that we would get a
            # total reward = `value` in the future. We took an action with log probability
            # of `log_prob` and ended up recieving a total reward = `ret`.
            # The actor must be updated so that it predicts an action that leads to
            # high rewards (compared to critic's estimate) with high probability.
            diff = ret - value
            actor_losses.append(-log_prob * diff)  # actor loss

            # The critic must be updated so that it predicts a better estimate of
            # the future rewards.
            critic_losses.append(
                huber_loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0))
            )

        # Backpropagation
        loss_actor = sum(actor_losses)
        grads_actor = tape.gradient(loss_actor, model_actor.trainable_variables)
        optimizer_actor.apply_gradients(zip(grads_actor, model_actor.trainable_variables))
        
        loss_critic = sum(critic_losses)
        grads_critic = tape.gradient(loss_critic, model_critic.trainable_variables)
        optimizer_critic.apply_gradients(zip(grads_critic, model_critic.trainable_variables))

        # Clear the loss and reward history
        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()

    # Log details
    episode_count += 1
    gboard.write_scalar('running_reward', running_reward, episode_count)
    if running_reward > 195:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break
        
gboard.write_env(env, model, step=episode_count)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 8081 (pid 6888), started 3 days, 18:45:29 ago. (Use '!kill 6888' to kill it.)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



KeyboardInterrupt: 

In [25]:
model_actor.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               640       
_________________________________________________________________
dense_16 (Dense)             (None, 2)                 258       
Total params: 898
Trainable params: 898
Non-trainable params: 0
_________________________________________________________________
